#### 2. Bilingual Summarization and Translation :

Following the initial summarization, the same website content is going to be summarized again, this time with an added layer of complexity: translation into French.
The translated summary maintained the core message while ensuring linguistic and cultural nuances were respected.
This bilingual approach highlighted the adaptability of the LLMs in handling multilingual tasks effectively.

#### Key takeaways :
- The model uses **mistral** which is a smaller,faster model and the one of the most popular open-source models available in Ollama.
- No API Key is needed: Ollama runs locally, so no API Key is required.
- Customization: You can adjust the max_tokens and temperature parameters to control the response lengh and creativity.

#### Alternative model in Ollama :
If you want to experiment with other models, you can replace **mistral** with one of the following:
- gemma3 : the current, most capable model that runs on a single GPU
- vicuna : a fine-tuned version of llama for conversational tasks
- llama3 : The most capable openly available LLM to date

To use a different model, make sure the model is isntalled loccally using Ollama. Simply pull it using ollama pull **model name** (e.g **ollama pull llama3**) and update the **model** constant in the code.


In [1]:
# IMPORT LABRAIRIES

import requests
import logging
from typing import Optional, Tuple

# CCONFIGURE LOGGING
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# CREATE CONSTANTS
OLLAMA_API_URL = 'http://localhost:11434/api/generate'
DEFAULT_MODEL = 'mistral'  # Can be changed to llama3 or other available models
SUMMARY_MAX_TOKENS = 350  # Increased for detailed bullet points
TRANSLATION_MAX_TOKENS = 400  # French translations often require more tokens

In [2]:
# CREATE FUNCTIONS

def call_ollama_api(prompt: str, model: str = DEFAULT_MODEL, 
                   max_tokens: int = SUMMARY_MAX_TOKENS, 
                   temperature: float = 0.5) -> Optional[str]:
    """
    Calls Ollama API with the given prompt and returns the response.
    No timeout to accommodate variable processing times.
    """
    try:
        payload = {
            'model': model,
            'prompt': prompt,
            'max_tokens': max_tokens,
            'temperature': temperature,
            'stream': False
        }
        
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        
        return response.json().get('response', '').strip()
    
    except requests.exceptions.RequestException as e:
        logger.error(f"API Error: {str(e)}")
    except Exception as e:
        logger.error(f"Unexpected Error: {str(e)}")
    return None

def generate_structured_summary(url: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Generates a structured English summary and French translation of a website.
    Returns tuple of (english_summary, french_translation)
    """
    # English prompt with strict formatting requirements
    english_prompt = f"""
    Analyze the content from {url} and provide a summary in 4-5 bullet points covering:
    1. The company's core purpose/mission
    2. Their main services or products
    3. Key value propositions/differentiators
    4. Primary target audience/customers
    
    Requirements:
    - Use concise bullet point format
    - Include only explicitly stated facts from the content
    - Omit any speculative or interpretive statements
    - Each bullet point should be 1-2 sentences maximum
    - Maintain professional tone
    
    Structure your response like this:
    • [Purpose]: ...
    • [Services]: ...
    • [Value]: ...
    • [Audience]: ...
    """
    
    # Generate English summary
    english_summary = call_ollama_api(english_prompt, max_tokens=SUMMARY_MAX_TOKENS)
    if not english_summary:
        return None, None
    
    # French translation prompt
    french_prompt = f"""
    Translate the following business summary from English to French:
    {english_summary}
    
    Requirements:
    - Maintain all bullet point formatting
    - Preserve professional business tone
    - Keep technical/business terms accurate
    - Ensure complete grammatical correctness
    """
    
    # Generate French translation
    french_translation = call_ollama_api(french_prompt, max_tokens=TRANSLATION_MAX_TOKENS)
    
    return english_summary, french_translation or "Translation failed"

def display_results(english: str, french: str) -> None:
    """Formats and displays the results clearly"""
    divider = "=" * 50
    logger.info("\n%s\nENGLISH SUMMARY:\n%s\n", divider, english)
    logger.info("\n%s\nFRENCH TRANSLATION:\n%s\n%s\n", divider, french, divider)

if __name__ == "__main__":
    website_url = "https://kamatechsolutions.com"
    
    logger.info("Generating structured summary and translation...")
    summary, translation = generate_structured_summary(website_url)
    
    if summary:
        display_results(summary, translation)
    else:
        logger.error("Failed to generate summary")

Generating structured summary and translation...

ENGLISH SUMMARY:
• Purpose: Kamatech Solutions is committed to delivering world-class IT solutions, fostering innovation, and enhancing business efficiency for their clients.
   • Services: They offer a wide range of services including software development, app development (iOS, Android), website design and development, digital marketing, managed IT services, and cloud solutions.
   • Value: Kamatech Solutions differentiates itself through agile methodologies, scalable solutions, 24/7 technical support, and a focus on delivering high-quality results that meet client needs and expectations.
   • Audience: Their primary target audience includes small to medium businesses (SMBs), startups, and established enterprises seeking reliable, innovative, and efficient IT services to drive their digital transformation.


FRENCH TRANSLATION:
• Objectif : Kamatech Solutions se engage à livrer des solutions IT de classe mondiale, à encourager l'innova

### **leveraging api/chat Endpoint**
Using **/api/chat** instead of **/api/generate** better leverages Ollama's chat capibilities while maintenaing the original requirements for structured business summaries and translations. 

When working with **LLM APIs**, the choice between **/api/chat** and **/api/generate endpoints** depends on your specific use case:
- it accepts conversation history with role (**system/user/assistant**)
- it usually supports system instructions, user messages, and assistant responses
- it improves prompt structure
- it maimtains context through message history
- Each API call includes the system message for consitency.
- it typically returns more natural-sounding responses suited for dialog

Note that the **api/chat** is best for chat capabilities or conversational applications where context and history matter.while **api/generate** is best for text generation without conversation history. It is often used for content creation, completion tasks, or summarization.

In [3]:
import requests
import logging
from typing import Optional, Tuple

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Constants
OLLAMA_API_URL = 'http://localhost:11434/api/chat'  # Using chat endpoint
DEFAULT_MODEL = 'llama3'  # Can be changed to mistral or other available models

def call_ollama_chat(messages: list, model: str = DEFAULT_MODEL) -> Optional[str]:
    """
    Calls Ollama's chat endpoint with conversation history
    """
    try:
        payload = {
            'model': model,
            'messages': messages,
            'stream': False
        }
        
        response = requests.post(OLLAMA_API_URL, json=payload)
        response.raise_for_status()
        
        return response.json()['message']['content'].strip()
    
    except requests.exceptions.RequestException as e:
        logger.error(f"API Error: {str(e)}")
    except Exception as e:
        logger.error(f"Unexpected Error: {str(e)}")
    return None

def generate_structured_summary(url: str) -> Tuple[Optional[str], Optional[str]]:
    """
    Generates structured summary and translation using chat endpoint
    """
    # System message to set behavior
    system_msg = {
        'role': 'system',
        'content': "You are a business analyst that extracts and summarizes information with perfect accuracy."
    }
    
    # English summary prompt
    english_prompt = {
        'role': 'user',
        'content': f"""Analyze {url} and provide a summary in this exact format:
        
        • [Purpose]: <company's core mission>
        • [Services]: <main offerings>
        • [Value]: <key differentiators>
        • [Audience]: <target customers>
        
        Rules:
        - Use only explicitly stated facts
        - No interpretation or speculation
        - Maximum 2 sentences per bullet
        - Maintain professional tone"""
    }
    
    # Generate English summary
    english_summary = call_ollama_chat([system_msg, english_prompt])
    if not english_summary:
        return None, None
    
    # French translation prompt
    french_prompt = {
        'role': 'user',
        'content': f"""Translate this business summary to French while preserving:
        
        {english_summary}
        
        Requirements:
        - Keep bullet point format
        - Maintain technical/business terms
        - Ensure perfect grammar
        - Use professional tone"""
    }
    
    # Generate French translation
    french_translation = call_ollama_chat([system_msg, french_prompt])
    
    return english_summary, french_translation or "Translation failed"

def display_results(english: str, french: str) -> None:
    """Formats and displays results"""
    divider = "=" * 60
    logger.info("\n%s\nENGLISH SUMMARY:\n%s\n", divider, english)
    logger.info("\n%s\nFRENCH TRANSLATION:\n%s\n%s\n", divider, french, divider)

if __name__ == "__main__":
    website_url = "https://kamatechsolutions.com"
    
    logger.info("Generating business summary and translation...")
    summary, translation = generate_structured_summary(website_url)
    
    if summary:
        display_results(summary, translation)
    else:
        logger.error("Failed to generate summary")

Generating business summary and translation...

ENGLISH SUMMARY:
Based on the website https://kamatechsolutions.com, here is a summary in the exact format:

• [Purpose]: Kamatech Solutions provides IT consulting services to help businesses achieve their goals through technology.
• [Services]: The company offers mainframe services, IT staffing solutions, and technology consulting services to its clients.

• [Value]: What sets Kamatech apart from other IT companies is their focus on delivering high-quality results and building long-term relationships with their clients. Their team of experts has extensive experience in the industry, ensuring that clients receive top-notch service.

• [Audience]: Kamatech Solutions primarily targets businesses looking for customized IT solutions to improve their operations, such as those in the financial services, healthcare, and government sectors.


FRENCH TRANSLATION:
Voici un résumé en français d'après le site web https://kamatechsolutions.com :

• [B